In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import math


In [21]:
random_tensor=torch.rand(4,4)
pos=torch.arange(0,100,device='cuda')

In [22]:
pos=pos.unsqueeze(dim=1)

In [23]:
pos

tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24],
        [25],
        [26],
        [27],
        [28],
        [29],
        [30],
        [31],
        [32],
        [33],
        [34],
        [35],
        [36],
        [37],
        [38],
        [39],
        [40],
        [41],
        [42],
        [43],
        [44],
        [45],
        [46],
        [47],
        [48],
        [49],
        [50],
        [51],
        [52],
        [53],
        [54],
        [55],
        [56],
        [57],
        [58],
        [59],
        [60],
        [61],
        [62],
        [63],
        [64],
        [65],
        [66],
        [67],
        [68],
        [69],
        [70],
      

In [24]:
pos.shape

torch.Size([100, 1])

embedding

In [ ]:
# 输入词汇表---->指定维度的embedding
class TokenEmbedding(nn.Embedding):
    def __init__(self,vocab_size,d_model):
        super(TokenEmbedding,self).__init__(vocab_size,d_model,padding_idx=1)


In [ ]:
# 编码可以改
class PositionEmbedding(nn.Module):
    def __init__(self, d_model, max_len, device):
        super().__init__(PositionEmbedding,self).__init__()
        self.encoding=torch.zeros(max_len,d_model,device=device)
        self.encoding.requires_grad=False
        pos=torch.arange(0,max_len,device=device)
        pos=pos.float().unsqueeze(dim=1)
        _2i=torch.arange(0,d_model,step=2,device=device).float()
        self.encoding[:,0::2]=torch.sin(pos/(10000**(_2i/d_model)))
        self.encoding[:,1::2]=torch.cos(pos/(10000**(_2i/d_model)))

    def forward(self,x):
        batch_size,seq_len=x.size()
        return self.encoding[:seq_len,:]



In [26]:
class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len, drop_prob,device):
        super(TransformerEmbedding,self).__init__()
        self.tok_embedding=TokenEmbedding(vocab_size,d_model)
        self.pos_embedding=PositionEmbedding(d_model,max_len,device)
        self.drop_out=nn.Dropout(p=drop_prob)

    def forward(self,x):
        tok_emb=self.tok_embedding(x)
        pos_emb=self.pos_embedding(x)
        return self.drop_out(tok_emb+pos_emb)
    
    

multi-attention

In [28]:
x=torch.rand(128,32,512)
# batchsize,seqlen,feature

d_model=512
n_head=8

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super(MultiHeadAttention,self).__init__()
        self.n_head=n_head
        self.d_model=d_model
        self.w_q=nn.Linear(d_model,d_model)
        self.w_k=nn.Linear(d_model,d_model)
        self.w_v=nn.Linear(d_model,d_model)
        self.w_combine=nn.Linear(d_model,d_model)
        self.softmax=nn.Softmax(dim=-1)
    
    def forward(self,q,k,v,mask=None):
        batch,time,dimension=q.shape
        n_d=self.d_model//self.n_head
        q,k,v=self.w_q(q),self.w_k(k),self.w_v(v)
        q=q.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        k=k.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        v=v.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        """
        变换前的形状：(batch, time, n_head, head_dim)
        变换后的形状：(batch, n_head, time, head_dim)
        torch.randn(2, 3, 4, 5).permute(0, 2, 1, 3).shape 会从 (2, 3, 4, 5) 变为 (2, 4, 3, 5)。
        """


        score=q@k.transpose(2,3)/math.sqrt(n_d)
        if mask is not None:
            score=score.masked_fill(mask==0,-10000)
        score=self.softmax(score)@v
        score=score.permute(0,2,1,3).contiguous().view(batch,time,dimension)
        output=self.w_combine(score)
        return output
    
attention=MultiHeadAttention(d_model,n_head)
out=attention(x,x,x)


In [39]:
out

tensor([[[ 0.1103, -0.2195, -0.0238,  ...,  0.2186,  0.2228, -0.2140],
         [ 0.1099, -0.2210, -0.0234,  ...,  0.2184,  0.2211, -0.2139],
         [ 0.1100, -0.2200, -0.0236,  ...,  0.2191,  0.2222, -0.2130],
         ...,
         [ 0.1108, -0.2209, -0.0247,  ...,  0.2186,  0.2219, -0.2135],
         [ 0.1098, -0.2208, -0.0235,  ...,  0.2183,  0.2224, -0.2135],
         [ 0.1099, -0.2199, -0.0244,  ...,  0.2184,  0.2220, -0.2131]],

        [[ 0.1003, -0.1952, -0.0111,  ...,  0.2119,  0.1832, -0.2234],
         [ 0.1004, -0.1967, -0.0115,  ...,  0.2122,  0.1842, -0.2241],
         [ 0.1002, -0.1962, -0.0111,  ...,  0.2111,  0.1839, -0.2249],
         ...,
         [ 0.1006, -0.1952, -0.0108,  ...,  0.2107,  0.1840, -0.2232],
         [ 0.0996, -0.1957, -0.0119,  ...,  0.2118,  0.1833, -0.2241],
         [ 0.1004, -0.1953, -0.0112,  ...,  0.2116,  0.1844, -0.2238]],

        [[ 0.0926, -0.2209, -0.0589,  ...,  0.1920,  0.1647, -0.1764],
         [ 0.0931, -0.2203, -0.0577,  ...,  0

In [38]:
out.shape

torch.Size([128, 32, 512])

layernorm--